# Natural Capital Modulation Tutorial

This notebook demonstrates the Natural Capital (NC) modulation feature, which adjusts the base EII score by ±0.05 based on three plot-level KPIs:

- **Biodiversity**: Natural habitat fraction from ESA WorldCover 10m
- **Soil**: Soil organic carbon relative to climate-zone reference (SoilGrids 250m)
- **Water**: Available water capacity relative to texture-typical maximum (SoilGrids 250m)

The NC score is the equally-weighted average of these three KPIs (0-1), which then modulates EII:
- NC = 0 → EII decreases by 0.05
- NC = 0.5 → EII unchanged  
- NC = 1 → EII increases by 0.05

## Setup

In [ ]:
import ee

from eii.client import (
    get_default_kpis,
    get_kpi_layers,
    get_modulated_eii,
    quicklook,
)

ee.Authenticate()
ee.Initialize(project="landler-open-data")

## Define test areas

We'll use three contrasting sites:
1. **Agricultural landscape** (Bavaria, Germany) - moderate natural habitat
2. **Protected area** (Swiss National Park) - high natural habitat
3. **Urban area** (Munich city center) - low natural habitat

In [ ]:
# Agricultural landscape in Bavaria
aoi_agricultural = ee.Geometry.Rectangle([10.5, 47.5, 10.7, 47.6])

# Swiss National Park
aoi_protected = ee.Geometry.Rectangle([10.1, 46.6, 10.3, 46.75])

# Munich city center
aoi_urban = ee.Geometry.Rectangle([11.55, 48.12, 11.6, 48.15])

_ = quicklook(aoi_agricultural, min_bbox_km=100, layer_color="yellow")

## Quick start: Get modulated EII with default KPIs

The simplest usage computes all KPIs from built-in GEE datasets.

In [ ]:
result = get_modulated_eii(
    geometry=aoi_agricultural,
    compute_default_kpis=True,
    stats=["mean"],
)

print("Agricultural landscape:")
print(f"  Base EII:       {result['values']['eii']['mean']:.4f}")
print(f"  Modulated EII:  {result['values']['eii_modulated']['mean']:.4f}")
print(f"  NC Score:       {result['values']['nc_score']['mean']:.4f}")
print(f"  Biodiversity:   {result['values']['biodiversity_kpi']['mean']:.4f}")
print(f"  Soil:           {result['values']['soil_kpi']['mean']:.4f}")
print(f"  Water:          {result['values']['water_kpi']['mean']:.4f}")

## Compare sites

Let's compare all three sites to see how the modulation varies.

In [ ]:
sites = {
    "Agricultural": aoi_agricultural,
    "Protected": aoi_protected,
    "Urban": aoi_urban,
}

results = []
for name, geometry in sites.items():
    result = get_modulated_eii(
        geometry=geometry,
        compute_default_kpis=True,
        stats=["mean"],
    )
    v = result["values"]
    results.append(
        {
            "Site": name,
            "Base EII": v["eii"]["mean"],
            "Modulated EII": v["eii_modulated"]["mean"],
            "NC Score": v["nc_score"]["mean"],
            "Biodiversity": v["biodiversity_kpi"]["mean"],
            "Soil": v["soil_kpi"]["mean"],
            "Water": v["water_kpi"]["mean"],
        }
    )

import pandas as pd

df = pd.DataFrame(results).set_index("Site")
df.round(4)

## Inspect individual KPIs

Use `get_default_kpis()` to get just the KPI values without computing the full modulation.

In [ ]:
kpis = get_default_kpis(aoi_protected)
print("Protected area KPIs:")
for name, value in kpis.items():
    print(f"  {name}: {value:.4f}")

## Use pre-computed KPI values

If you have your own KPI measurements (e.g., from field data), pass them directly.

In [ ]:
# Scenario: Well-managed farm with high soil carbon and water retention
custom_kpis = {
    "biodiversity": 0.35,  # 35% natural habitat
    "soil": 0.85,  # High SOC relative to reference
    "water": 0.78,  # Good water holding capacity
}

result = get_modulated_eii(
    geometry=aoi_agricultural,
    kpis=custom_kpis,
    stats=["mean"],
)

v = result["values"]
print(f"Base EII:      {v['eii']['mean']:.4f}")
print(f"NC Score:      {v['nc_score']['mean']:.4f}")
print(f"Modulated EII: {v['eii_modulated']['mean']:.4f}")
print(f"Modulation:    {v['eii_modulated']['mean'] - v['eii']['mean']:+.4f}")

## Custom KPI weighting

Override the default equal weighting (1/3 each) with custom weights.

In [ ]:
# Emphasize biodiversity (50%), de-emphasize soil/water (25% each)
result = get_modulated_eii(
    geometry=aoi_agricultural,
    compute_default_kpis=True,
    kpi_weights={
        "biodiversity": 0.5,
        "soil": 0.25,
        "water": 0.25,
    },
    stats=["mean"],
)

v = result["values"]
print(f"NC Score (biodiversity-weighted): {v['nc_score']['mean']:.4f}")

## Adjust biodiversity threshold

The default biodiversity threshold is 50% (i.e., 50% natural habitat = score 1.0).
For intensive agricultural areas, you might use a lower threshold like 30%.

In [ ]:
# Default: 50% threshold
result_50 = get_modulated_eii(
    geometry=aoi_agricultural,
    compute_default_kpis=True,
    biodiversity_max_threshold=0.50,
    stats=["mean"],
)

# Stricter: 30% threshold
result_30 = get_modulated_eii(
    geometry=aoi_agricultural,
    compute_default_kpis=True,
    biodiversity_max_threshold=0.30,
    stats=["mean"],
)

print("Biodiversity KPI comparison:")
print(f"  50% threshold: {result_50['values']['biodiversity_kpi']['mean']:.4f}")
print(f"  30% threshold: {result_30['values']['biodiversity_kpi']['mean']:.4f}")

## Use custom raster layers

Provide your own ee.Image layers for each KPI (must be normalized 0-1).

In [ ]:
# Example: Create custom biodiversity layer from NDVI threshold
# (This is just for demonstration - use appropriate data for your use case)

# Get default layers for soil and water
default_layers = get_kpi_layers(aoi_agricultural)

# Create a simple custom biodiversity proxy from Sentinel-2 NDVI
s2 = (
    ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
    .filterBounds(aoi_agricultural)
    .filterDate("2023-06-01", "2023-08-31")
    .filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE", 20))
    .median()
)

ndvi = s2.normalizedDifference(["B8", "B4"])
# Map NDVI 0.2-0.8 to 0-1 as a simple "greenness" proxy
custom_biodiv = ndvi.subtract(0.2).divide(0.6).clamp(0, 1).rename("biodiversity_kpi")

# Use custom biodiversity with default soil/water
result = get_modulated_eii(
    geometry=aoi_agricultural,
    kpi_layers={
        "biodiversity": custom_biodiv,
        "soil": default_layers["soil"],
        "water": default_layers["water"],
    },
    stats=["mean"],
)

print(f"Custom biodiversity KPI: {result['values']['biodiversity_kpi']['mean']:.4f}")

## Output as GeoDataFrame

Use `output_format="geodataframe"` for integration with GeoPandas workflows.

In [ ]:
gdf_result = get_modulated_eii(
    geometry=aoi_agricultural,
    compute_default_kpis=True,
    stats=["mean", "std"],
    output_format="geodataframe",
)
gdf_result

## Understanding the modulation formula

The modulation is applied as:
```
EII_final = EII_base + (NC - 0.5) * 0.1
```

This means:
- If NC = 0.5 (average), EII is unchanged
- If NC = 1.0 (excellent), EII increases by 0.05
- If NC = 0.0 (poor), EII decreases by 0.05

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Visualize modulation behavior
eii_base = np.linspace(0, 1, 100)
nc_scores = [0, 0.25, 0.5, 0.75, 1.0]

fig, ax = plt.subplots(figsize=(8, 6))

for nc in nc_scores:
    eii_mod = np.clip(eii_base + (nc - 0.5) * 0.1, 0, 1)
    ax.plot(eii_base, eii_mod, label=f"NC = {nc:.2f}")

ax.plot([0, 1], [0, 1], "k--", alpha=0.3, label="No modulation")
ax.set_xlabel("Base EII")
ax.set_ylabel("Modulated EII")
ax.set_title("EII Modulation by Natural Capital Score")
ax.legend()
ax.set_xlim(0, 1)
ax.set_ylim(0, 1)
ax.set_aspect("equal")
plt.tight_layout()

## Summary

Key functions for Natural Capital modulation:

| Function | Purpose |
|----------|--------|
| `get_modulated_eii()` | Main entry point - returns base EII, modulated EII, NC score, and KPIs |
| `get_default_kpis()` | Get just the KPI values (biodiversity, soil, water) |
| `get_kpi_layers()` | Get KPI raster layers (ee.Image) for visualization |
| `get_nc_score()` | Calculate NC score from KPI inputs |

Three ways to provide KPIs:
1. `compute_default_kpis=True` - Use built-in GEE datasets (ESA WorldCover, SoilGrids)
2. `kpis={...}` - Pre-computed float values (0-1)
3. `kpi_layers={...}` - Custom ee.Image layers (0-1)